In [16]:
import nnvm.compiler
import nnvm.symbol as sym
import tvm
import numpy as np
from   tvm.contrib import graph_runtime, util

In [17]:
x = sym.Variable("x")
y = sym.Variable("y")
z = sym.elemwise_add(x, sym.sqrt(y))
compute_graph = nnvm.graph.create(z)
print("-------compute graph-------")
print(compute_graph.ir())

-------compute graph-------
Graph(%x, %y) {
  %2 = sqrt(%y)
  %3 = elemwise_add(%x, %2)
  ret %3
}


In [18]:
shape = (4,)
deploy_graph, lib, params = nnvm.compiler.build(
    compute_graph, target='opencl', shape={"x": shape}, dtype="float32")

In [19]:
print(lib.imported_modules[0].get_source())

__kernel void fuse_sqrt_elemwise_add_kernel0(__global float* restrict tensor, __global float* restrict input1, __global float* restrict input0) {
  if (((int)get_local_id(0)) < 4) {
    tensor[((int)get_local_id(0))] = (input1[((int)get_local_id(0))] + sqrt(input0[((int)get_local_id(0))]));
  }
}




In [20]:
module = graph_runtime.create(deploy_graph, lib, tvm.opencl())
x_np = np.array([1, 2, 3, 4]).astype("float32")
y_np = np.array([4, 9, 16, 25]).astype("float32")
module.set_input(x=x_np, y=y_np)
module.run()
out = module.get_output(0, out=tvm.nd.empty(shape))
print(out.asnumpy())


[3. 5. 7. 9.]


In [24]:
shape = (5,)
deploy_graph, lib, params = nnvm.compiler.build(
    compute_graph, target='llvm', shape={"x": shape}, dtype="float32")


None


In [22]:
module = graph_runtime.create(deploy_graph, lib, tvm.cpu())
x_np = np.array([1, 2, 3, 4, 5]).astype("float32")
y_np = np.array([4, 9, 16, 25, 36]).astype("float32")
module.set_input(x=x_np, y=y_np)
module.run()
out = module.get_output(0, out=tvm.nd.empty(shape))
print(out.asnumpy())

[ 3.  5.  7.  9. 11.]


In [25]:
lib.export_library('den.so')

with open("deploy.json", "w") as fo:
    fo.write(deploy_graph.json())
    
